In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.90 sec - Loss 0.243613 - ACC 77.61% - ACC Mean 77.61% - AUC 85.66% - AUC Mean 85.66% - Deter 000
Ite 00050 - 0.95 sec - Loss 0.122674 - ACC 78.07% - ACC Mean 77.67% - AUC 84.44% - AUC Mean 84.01% - Deter 047
Ite 00053 - 1.23 sec - Loss 0.120377 - ACC 77.91% - ACC Mean 77.69% - AUC 84.30% - AUC Mean 83.88% - Deter 050
Early stop ite 53, rollback to correction of ite 3, whith acc of 77.07% and auc of 86.4%
OUR METHOD RUN 1/10 - acc: 77.07% +- 0.0% - auc: 86.4% +- 0.0%
Ite 00000 - 0.95 sec - Loss 0.247092 - ACC 78.02% - ACC Mean 78.02% - AUC 85.75% - AUC Mean 85.75% - Deter 000
Ite 00050 - 1.04 sec - Loss 0.130605 - ACC 78.14% - ACC Mean 78.18% - AUC 84.38% - AUC Mean 84.45% - Deter 050
Early stop ite 50, rollback to correction of ite 0, whith acc of 78.02% and auc of 85.75%
OUR METHOD RUN 2/10 - acc: 78.02% +- 0.0% - auc: 85.75% +- 0.0%
Ite 00000 - 1.03 sec - Loss 0.218782 - ACC 75.94% - ACC Mean 75.94% - AUC 85.19% - AUC Mean 85.19% - Deter 000
Ite 00050 - 1.05 sec - Loss 

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 64.11% +- 0.0% - auc: 73.54% +- 0.0%
MEAN - acc: 77.3% +- 0.0% - auc: 85.09% +- 0.0%
KNN - acc: 68.83% +- 0.0% - auc: 78.94% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:52<00:00, 95.35it/s]



GAIN RUN 1/10 - acc: 61.99% +- 2.48% - auc: 72.96% +- 2.01%



100%|██████████| 5000/5000 [00:57<00:00, 87.58it/s]



GAIN RUN 2/10 - acc: 62.01% +- 1.65% - auc: 74.38% +- 1.59%



100%|██████████| 5000/5000 [00:55<00:00, 89.75it/s]


...


GAIN RUN 8/10 - acc: 60.93% +- 2.57% - auc: 73.7% +- 2.94%



100%|██████████| 5000/5000 [01:00<00:00, 82.77it/s]



GAIN RUN 9/10 - acc: 64.51% +- 2.0% - auc: 73.87% +- 4.06%



100%|██████████| 5000/5000 [01:01<00:00, 80.67it/s]



GAIN RUN 10/10 - acc: 68.93% +- 1.69% - auc: 77.19% +- 1.33%

GAIN GLOBAL - acc: 63.89% +- 2.21% - auc: 74.22% +- 1.11%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 76.55% +- 0.0% - auc: 81.39% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 1/10 - acc: 71.51% +- 2.58% - auc: 77.34% +- 4.75%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 2/10 - acc: 72.43% +- 3.05% - auc: 81.76% +- 2.9%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 3/10 - acc: 73.81% +- 1.69% - auc: 81.77% +- 2.76%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7

MISSFOREST RUN 4/10 - acc: 72.66% +- 1.45% - auc: 79.61% +- 4.0%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 5/10 - acc: 73.99% +- 2.57% - auc: 81.68% +- 2.71%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 6/10 - acc: 73.24% +- 3.83% - auc: 83.16% +- 3.25%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 79.56% +- 3.17% - auc: 85.88% +- 1.83%
SOFTIMPUTE RUN 2/10 - acc: 76.79% +- 2.22% - auc: 85.11% +- 2.75%
SOFTIMPUTE RUN 3/10 - acc: 78.06% +- 1.38% - auc: 84.35% +- 1.73%
SOFTIMPUTE RUN 4/10 - acc: 77.23% +- 1.86% - auc: 84.55% +- 2.67%
SOFTIMPUTE RUN 5/10 - acc: 77.27% +- 1.06% - auc: 86.42% +- 1.57%
SOFTIMPUTE RUN 6/10 - acc: 76.07% +- 2.04% - auc: 84.31% +- 3.03%
SOFTIMPUTE RUN 7/10 - acc: 77.2% +- 1.53% - auc: 85.33% +- 2.22%
SOFTIMPUTE RUN 8/10 - acc: 77.22% +- 1.5% - auc: 84.47% +- 2.84%
SOFTIMPUTE RUN 9/10 - acc: 77.29% +- 1.77% - auc: 84.66% +- 2.35%
SOFTIMPUTE RUN 10/10 - acc: 75.75% +- 2.24% - auc: 83.74% +- 3.96%

SOFTIMPUTE GLOBAL - acc: 77.24% +- 0.99% - auc: 84.88% +- 0.77%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7690

SINKHORN RUN 1/10 - acc: 75.83% +- 1.86% - auc: 82.06% +- 2.94%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7215

SINKHORN RUN 2/10 - acc: 75.86% +- 1.5% - auc: 82.55% +- 1.16%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.7614

SINKHORN RUN 3/10 - acc: 78.64% +- 2.26% - auc: 84.9% +- 1.13%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8255

SINKHORN RUN 4/10 - acc: 74.64% +- 2.86% - auc: 83.43% +- 0.98%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8721

SINKHORN RUN 5/10 - acc: 74.37% +- 1.77% - auc: 83.73% +- 1.04%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8616

SINKHORN RUN 6/10 - acc: 75.65% +- 2.24% - auc: 81.39% +- 2.78%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.8772

SINKHORN RUN 7/10 - acc: 75.41% +- 1.68% - auc: 83.84% +- 4.7%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.9172

SINKHORN RUN 8/10 - acc: 75.11% +- 2.31% - auc: 82.7

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 76.16% +- 2.25% - auc: 84.22% +- 1.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 74.0% +- 2.89% - auc: 82.24% +- 1.04%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 74.92% +- 2.95% - auc: 82.1% +- 2.45%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 76.11% +- 2.31% - auc: 83.9% +- 1.52%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 75.67% +- 2.68% - auc: 83.53% +- 2.4%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 74.47% +- 2.72% - auc: 82.63% +- 3.24%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 75.48% +- 2.16% - auc: 82.0% +- 3.19%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 75.04% +- 2.08% - auc: 83.1% +- 1.06%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 74

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bet